# Data Splitter 
Daphne is turning the original Python code into a jupyter notebook!

This program is designed to separate the larger chunks of data that we receive from SQMs and separate them into night-by-night files of data

Try changing the directory set up so these are all in a descriptive folder

In [19]:
# imports
import os
import os.path
import datetime

#### User Input: designate the home directory (This is where the SQM bulk data is stored)
take all files from a specified directory and separate the larger file into a night-by-night basis 


In [20]:
# home_dir = "C:\\Vayujeet\\research\\LP\\Magruder Hall"
# home_dir ="C:\\LightPollution\\Grand Gulf"
home_dir = r"C:\Users\djzak\OneDrive\Documents\LightPollution\Grand Gulf"

In [21]:
# loop through the home directory, retrieve SQM location, and get rid of items that don't start with a date

for filename in os.listdir(home_dir):

    # print the files in directory
    f = os.path.join(home_dir, filename)
    if filename.endswith(".dat"):
        print(f)

        #infile = open("C:\\Vayujeet\\research\\LP\\Ozark -- Luke\\Raw Data\\20201016_135422_Luke.dat",'r')
        #infile = open("20201016_135422_Luke.dat",'r')
        infile = open(f)
        
        data = infile.readlines()

        # retrieve the location of this sqm
        location_line = data[7]  ## the location name always appears in line 8


        location_list = location_line.split() # split the line by words
        for i in range(2):  ## the first 3 items (words in the line) are #, Location, name:
            location_list.pop(0)
        location = "" 

        # Now, the following words  
        for i in range(len(location_list)):
            if i != 0:
                location += ' '
                location += location_list[i]

        
        ## get rid of everything that doesn't start with a date
        new_data = []
        for item in data:
            if item[0:2] == '20':  ## This is when it starts having a date first
                new_data.append(item)


        ## make a list with each night as a sublist
        night_list = []
        night = []
        my_time = datetime.datetime.fromisoformat(new_data[0][24:40])

        # (CHECKPOINT) - ask if this is the correct convention:
        # we want all measurements from the same night to be considered from the same day.
        # our convention is to designate all objects as the date of the evening
        # associated with the observations.
        # So, a 'new day' starts at noon
        
        if my_time.hour >= 12:
            current_day = datetime.datetime(my_time.year,my_time.month,my_time.day,12,1)
            current_day += datetime.timedelta(days=1)
        else:
            current_day = datetime.datetime(my_time.year,my_time.month,my_time.day,12,1)
        for i in new_data:
            time = datetime.datetime.fromisoformat(i[24:40]) ## picks out the local time
            if time > current_day:
                night_list += [night]
                night = [i]
                current_day += datetime.timedelta(days=1)
            else:
                night += [i]
        night_list += [night]  ## get the last partial day


        ## get rid of all the crap I don't want
        ## leave date;local time;SQM
        data_point = ""
        one_night = [] 
        all_nights= []
        for item in night_list:
            for line in item:
                time = line[35:40] ## the location of the local time
                date = line[24:34] ## the location of the local date
                sqm = "" ## getting SQM is more complicated, because of diff lengths
                reverse_line = line[::-1] ## save time, SQM is near back
                semicolons = 0
                for letter in reverse_line:
                    if letter == ";":
                        semicolons += 1
                    elif semicolons == 1:
                        sqm += letter
                my_sqm = sqm[::-1]
                data_point += date + ";" + time + ";"  + my_sqm
                one_night += [data_point]
                data_point = ""
            all_nights += [one_night]
            one_night = []


    
        ## Write each night onto a file
        
        ##save_path = "C:/Users/letsp/Documents/Summer Research 2020/Daily Data"
        ###save_path = "mnt/c/Vayujeet/Research/LP/Daily_Data/"
        ##save_path += "/" +  location
        #save_path = op_dir_name + " " + location

        # make a new folder for the split data if it doesn't already exist
        # put in a folder named as save_path (based on location)
        # checkpoint: replaced lstrip with strip
        save_path = location.lstrip()
        
        if not os.path.exists(save_path):
	        os.makedirs(save_path)

        print(save_path)
        for night in all_nights:
            # file_name = op_dir_name + " " +  location + night[0][:10]
            # file_name = night[0][:10]
            file_name = location.lstrip() + night[0][:10]
            
            
            full_name = os.path.join(save_path, file_name + ".txt")
            # full_name = os.path.join(file_name + ".txt")
            my_file = open(full_name, "w") ## may need to change this for if the files haven't been made yet
            for line in night:
                my_file.write(line + "\n")
            my_file.close()

C:\Users\djzak\OneDrive\Documents\LightPollution\Grand Gulf\20201223_145419_AH073XBP.dat
Grand Gulf State Park
C:\Users\djzak\OneDrive\Documents\LightPollution\Grand Gulf\20210107_143512_AH073XBP.dat
Grand Gulf State Park
C:\Users\djzak\OneDrive\Documents\LightPollution\Grand Gulf\20210324_085017_AH073XBP.dat
Grand Gulf State Park
C:\Users\djzak\OneDrive\Documents\LightPollution\Grand Gulf\20210508_083932_AH073XBP.dat
Grand Gulf State Park
C:\Users\djzak\OneDrive\Documents\LightPollution\Grand Gulf\20210819_150031_AH073XBP.dat
Grand Gulf State Park
C:\Users\djzak\OneDrive\Documents\LightPollution\Grand Gulf\20211214_155522_AH073XBP.dat
Grand Gulf State Park
